# In-class exercises 6

## Objectives: use the text to predict the political orientation of the news articles
The target $y$ is given by the `biais` column. 

You can use all remaining information in the dataset to predict the target.

In [ ]:
import pandas as pd


In [20]:
df = pd.read_csv("Political_Bias.csv")

In [21]:
df.head()

,Title,Link,Text,Source,Bias
0,John Boltonâ€™s Advice for the â€˜Continuing T...,https://thedispatch.com/article/john-boltons-a...,"With just days until his inauguration, it woul...",TheDispatch,lean right
1,Why Shrubland Makes Southern Californiaâ€™s Wi...,https://thedispatch.com/newsletter/techne/why-...,"Welcome back to Techne! In Star Trek, writers ...",TheDispatch,lean right
2,Los Angelesâ€™ Destruction Was Fueled by Bad P...,https://thedispatch.com/newsletter/capitolism/...,"As fire still rages in the Los Angeles area, I...",TheDispatch,lean right
3,Acquiring Greenland Is a Good Idea. Threatenin...,https://thedispatch.com/article/acquiring-gree...,"Whether it was Will Rogers or Tony Soprano, th...",TheDispatch,lean right
4,Assessing Trumpâ€™s Expansionist Designs,https://thedispatch.com/newsletter/morning/ass...,"Happy Friday! If anyone was wondering, Belgium...",TheDispatch,lean right


In [29]:
def fix_common_misencoding(text):
    replacements = {
        "â€™": "’",  # Fix curly apostrophe
        "â€˜": "‘",  # Fix left curly quote
        "â€”": "—",  # Fix em dash (if present)
        "â€œ": "“",  # Fix left double quote (if present)
        "â€": "”",   # Fix right double quote (if present)
    }
    for wrong, correct in replacements.items():
        text = text.replace(wrong, correct)
    return text

In [31]:
df['Title'] = df['Title'].apply(fix_common_misencoding)

In [33]:
df.head()

,Title,Link,Text,Source,Bias
0,John Bolton’s Advice for the ‘Continuing Turmo...,https://thedispatch.com/article/john-boltons-a...,"With just days until his inauguration, it woul...",TheDispatch,lean right
1,Why Shrubland Makes Southern California’s Wild...,https://thedispatch.com/newsletter/techne/why-...,"Welcome back to Techne! In Star Trek, writers ...",TheDispatch,lean right
2,Los Angeles’ Destruction Was Fueled by Bad Pol...,https://thedispatch.com/newsletter/capitolism/...,"As fire still rages in the Los Angeles area, I...",TheDispatch,lean right
3,Acquiring Greenland Is a Good Idea. Threatenin...,https://thedispatch.com/article/acquiring-gree...,"Whether it was Will Rogers or Tony Soprano, th...",TheDispatch,lean right
4,Assessing Trump’s Expansionist Designs,https://thedispatch.com/newsletter/morning/ass...,"Happy Friday! If anyone was wondering, Belgium...",TheDispatch,lean right


In [34]:
df.shape

(3458, 5)